# Create Training datasets
The process for creating the initial training datasets for the broad topicCategories NOT included in litcovid are as follows:
1. Brainstorm keywords that should be relatively SPECIFIC for that topicCategory
2. Perform queries to the APIs to retrieve all ids returned by each topicCategory-specific keyword
3. Retrieve all keywords for each id
4. Create frequency tables of all keywords
5. Take top ~5% of most frequent keywords
6. Take top 100 most frequent keywords from litcovid topics
8. Use the keywords to remove more generic terms like 'COVID19'
9. manually inspect for specific-enough terms for inclusion into original keyword list
10. Repeat process, adding keywords to grow the training dataset, or removing keywords which are causing the dataset to become less specific

## Create Dataset based on initial curated keyword lists

In [1]:
import os
import requests
import json
import pandas as pd
from pandas import read_csv
from datetime import datetime
#import pickle
#import sklearn
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score


In [2]:
DATAPATH = 'data/'
KEYPATH = os.path.join(DATAPATH,'keywords/')
KEYFILES = os.listdir(KEYPATH)
RESULTSPATH = 'results/'

In [ ]:
## 'https://api.outbreak.info/resources/resource/query?q=loneliness&filter=@type:Publication&fields=_id'

### Functions for using the outbreak api

In [3]:
#### Pull ids from a json file
def get_ids_from_json(jsonfile):
    idlist = []
    for eachhit in jsonfile["hits"]:
        if eachhit["_id"] not in idlist:
            idlist.append(eachhit["_id"])
    return(idlist)

def fetch_query_size(query):
    pubmeta = requests.get('https://api.outbreak.info/resources/query?q=(("'+query+'") AND (@type:Publication))&size=0&aggs=@type')
    pubjson = json.loads(pubmeta.text)
    pubcount = int(pubjson["facets"]["@type"]["total"])
    return(pubcount)

#### Ping the API and get all the ids for a specific source and scroll through the source until number of ids matches meta
def get_query_ids(query):
    query_size = fetch_query_size(query)
    r = requests.get('https://api.outbreak.info/resources/query?q=(("'+query+'") AND (@type:Publication))&fields=_id&fetch_all=true')
    response = json.loads(r.text)
    idlist = get_ids_from_json(response)
    try:
        scroll_id = response["_scroll_id"]
        while len(idlist) < query_size:
            r2 = requests.get('https://api.outbreak.info/resources/query?q=(("'+query+'") AND (@type:Publication))&fields=_id&fetch_all=true&scroll_id='+scroll_id)
            response2 = json.loads(r2.text)
            idlist2 = set(get_ids_from_json(response2))
            tmpset = set(idlist)
            idlist = tmpset.union(idlist2)
            try:
                scroll_id = response2["_scroll_id"]
            except:
                print("no new scroll id")
        return(idlist)
    except:
        return(idlist)


def batch_fetch_kw(idlist):
    ## Break the list of ids into smaller chunks so the API doesn't fail the post request
    runs = round((len(idlist))/100,0)
    i=0 
    separator = ','
    ## Create dummy dataframe to store the meta data
    keywordsdf = pd.DataFrame(columns = ['_id','keywords'])
    while i < runs+1:
        if len(idlist)<100:
            sample = idlist
        elif i == 0:
            sample = idlist[i:(i+1)*100]
        elif i == runs:
            sample = idlist[i*100:len(idlist)]
        else:
            sample = idlist[i*100:(i+1)*100]
        sample_ids = separator.join(sample)
        r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': sample_ids, 'scopes': '_id', 'fields': 'keywords'})
        if r.status_code == 200:
            rawresult = pd.read_json(r.text)
            cleanresult = rawresult[['_id','keywords']].loc[rawresult['_score']==1].copy()
            cleanresult.drop_duplicates(subset='_id',keep="first", inplace=True)
            keywordsdf = pd.concat((keywordsdf,cleanresult))
        i=i+1
    return(keywordsdf)




### Functions to load the initial search terms and litcovid dataset

In [4]:
def load_search_terms():
    keyword_dict = {}
    for eachfile in KEYFILES:
        filename = eachfile.split('.')[0]
        keywords = []
        with open(os.path.join(KEYPATH,eachfile),'r') as readfile:
            for eachline in readfile:
                keywords.append(eachline.strip())
        keyword_dict[filename]=keywords
    return(keyword_dict)


def load_category_ids():
    keyword_dict = load_search_terms()
    cat_dict = {}
    for category in keyword_dict.keys():
        allids = []
        idlist = []
        keywordlist = keyword_dict[category]
        for eachkey in keywordlist:
            idlist = get_query_ids(eachkey)
            allids = list(set(allids).union(set(idlist)))
        cat_dict[category]=allids
    return(cat_dict)


def load_litcovid_ids():
    litcovid = read_csv(os.path.join(DATAPATH,'litcovidtopics.tsv'),delimiter='\t',header=0,index_col=0)
    topics = litcovid['topicCategory'].unique().tolist()
    litcovid_dict={}
    for eachtopic in topics:
        tmplist = litcovid['_id'].loc[litcovid['topicCategory']==eachtopic].tolist()
        litcovid_dict[eachtopic]=tmplist
    return(litcovid_dict)

### Functions for creating keyword frequency tables for evaluation (and manual inspection)

Expand the search based on keywords that appeared from the first round of searching. Remove generic terms (such as COVID-19) by generating the top most frequent keywords from LitCovid. 

1. For the first pass, create a frequency table, keeping only the top 5% non-generic keywords for evaluation
2. For the second pass (with an expanded search term list), keep only the top 1% of terms for evaluation as 5% may be too large a set to evaluate
3. After reviewing the 1% from the second pass, a 3rd pass was determined to be unnecessary

In [5]:
#### Obtain the most frequent (ie- generic) keywordss from litcovidtopics
#### Default is top 100 most generic, but this can be adjusted
def get_generic_terms(top=100):
    litcovid = read_csv(os.path.join(DATAPATH,'litcovidtopics.tsv'),delimiter='\t',header=0,index_col=0)
    idlist = litcovid['_id'].unique().tolist()
    tmpdf = batch_fetch_kw(idlist)
    kwdf = tmpdf.set_index('_id').keywords.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index()
    kwdf.rename(columns={0:'keywords'},inplace=True)
    kwdf['keywords']=kwdf['keywords'].astype(str).str.lower()
    kwfreq = kwdf.groupby('keywords').size().reset_index(name='count')
    kwfreq.sort_values('count',ascending=False,inplace=True)
    return(kwfreq.head(n=top).copy())


def get_kw_freq(topicCategory):
    cat_dict = load_category_ids()
    litcovid_dict = load_litcovid_ids()
    if topicCategory in cat_dict.keys():
        idlist = cat_dict[topicCategory]
    else:
        idlist = litcovid_dict[topicCategory]
    tmpdf = batch_fetch_kw(idlist)
    kwdf = tmpdf.set_index('_id').keywords.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index()
    kwdf.rename(columns={0:'keywords'},inplace=True)
    kwdf['keywords']=kwdf['keywords'].astype(str).str.lower()
    kwfreq = kwdf.groupby('keywords').size().reset_index(name='count')
    kwfreq.sort_values('count',ascending=False,inplace=True)
    return(kwfreq)


#### Set the count of the generic keywords to -1, and re-sort the list
def negate_genkw(kwfreq,genkw):
    genkwlist = genkw['keywords'].tolist()
    kwfreq['count'].loc[kwfreq['keywords'].isin(genkwlist)]=-1
    kwfreq.sort_values('count',ascending=False,inplace=True)
    return(kwfreq)


#### Export the most frequent x% of keywords
#### Note that 5(%) is default, but any number can be used
#### Also, the topicCategory is required for filename purposes
def export_most_freqkw(kwfreq,topicCategory,toppercent=5):
    export_num = int(round(len(kwfreq)*toppercent/100,0))
    exportdf = kwfreq.head(n=export_num)
    filepath = RESULTSPATH+'wordfrequencies/'
    filename = topicCategory+'_'+str(toppercent)+'_percent_most_freq_kw.txt'
    exportdf.to_csv(os.path.join(filepath,filename),sep='\t',header=True)

### Functions for comparing search results from Outbreak and LitCovid APIs to generate the training dataset

Also, hit the LitCovid API to generate initial list of pmids. Then select the pmids that were returned by BOTH the Outbreak API and the LitCovid API to get the ones that are more likely to be on-topic.

Eg- 'https://www.ncbi.nlm.nih.gov/research/coronavirus-api/export/tsv?text=%22emission%20reduction%22&filters=%7B%7D'

In [6]:
## Search litcovid for a term and retrieve the pmids
def search_litcovid_ids(searchterm):
    check_litcovid = requests.get('https://www.ncbi.nlm.nih.gov/research/coronavirus-api/export/tsv?text="'+searchterm+'"&filters={}')
    litcovid_data = check_litcovid.text.split('\n')[34:]
    pmids = []
    for line in litcovid_data:
        if line.startswith('#') or line.startswith('p'):
            continue
        pmids.append(line.split('\t')[0])
    cleanpmids = ['pmid'+x for x in pmids if x != ""]
    return(cleanpmids)

## load the list of search terms for each topicCategory and push it to either outbreak (default) or litcovid
## returns an id list
def category_id_check(source='outbreak'):
    keyword_dict = load_search_terms()
    allids = []
    if source == 'litcovid':
        for category in keyword_dict.keys():  
            keywordlist = keyword_dict[category]
            for eachkey in keywordlist:
                idlist = search_litcovid_ids(eachkey)
                allids.append({'category':category,'searchterm':eachkey,'ids':idlist})
        idcheck = pd.DataFrame(allids)
    else:
        for category in keyword_dict.keys():  
            keywordlist = keyword_dict[category]
            for eachkey in keywordlist:
                idlist = get_query_ids(eachkey)
                allids.append({'category':category,'searchterm':eachkey,'ids':idlist})
        idcheck = pd.DataFrame(allids)
    return(idcheck)


## Pull the id lists after search outbreak and litcovid and compare them
## keep only ids in common for training purposes
## Note, this will remove all preprints from the training set since litcovid does not have them
def get_in_common_ids():
    outbreakids = category_id_check()
    litcovidids = category_id_check(source='litcovid')
    mergedf = outbreakids.merge(litcovidids,on=(['category','searchterm']),how='outer')
    i=0
    tmplist = []
    while i <len(mergedf):
        idsincommon = list(set(mergedf.iloc[i]['ids_x']).intersection(set(mergedf.iloc[i]['ids_y'])))
        tmplist.append({'category':mergedf.iloc[i]['category'],
                        'searchterm':mergedf.iloc[i]['searchterm'],
                        'len_ids_x':len(mergedf.iloc[i]['ids_x']),
                        'len_ids_y':len(mergedf.iloc[i]['ids_y']),
                        'len_clean_ids':len(idsincommon),
                        'clean_ids':idsincommon})
        i=i+1
    cleandf = pd.DataFrame(tmplist)
    return(cleandf)   


def generate_training_dict(savefile = False):
    cleandf = get_in_common_ids()
    training_dict = {}
    for eachcat in cleandf['category'].unique().tolist(): 
        j=0
        tmpdf = cleandf.loc[cleandf['category']==eachcat]
        allids = []
        while j<len(tmpdf):
            allids = list(set(allids).union(set(tmpdf.iloc[j]['clean_ids'])))
            j=j+1
        training_dict[eachcat]=allids
    if savefile == False:
        return(training_dict)
    else:
        with open(os.path.join(RESULTSPATH,"training_dict.json"), "w") as outfile: 
            json.dump(training_dict, outfile)        

def transform_training_dict(training_dict):
    trainingdf = pd.DataFrame(columns=['_id','topicCategory'])
    for eachcat in training_dict.keys():
        idlist = pd.DataFrame(training_dict[eachcat])
        idlist.rename(columns={0:'_id'},inplace=True)
        idlist['topicCategory']=eachcat
        trainingdf = pd.concat((trainingdf,idlist),ignore_index=True)
    return(trainingdf)

#### Main script for generating training dataset (assuming manual expansion of search terms was performed)

In [ ]:
training_dict = generate_training_dict()
trainingdf = transform_training_dict(training_dict)
print(trainingdf['topicCategory'].unique().tolist(),len(trainingdf))
trainingdf.to_csv(os.path.join(RESULTSPATH,'training_ids.tsv'),sep='\t',header=True)

## Export training set as topics to be added


In [7]:
def clean_results(allresults):
    counts = allresults.groupby('_id').size().reset_index(name='counts')
    duplicates = counts.loc[counts['counts']>1]
    singles = counts.loc[counts['counts']==1]
    dupids = duplicates['_id'].unique().tolist()
    tmplist = []
    for eachid in dupids:
        catlist = allresults['topicCategory'].loc[allresults['_id']==eachid].tolist()
        tmplist.append({'_id':eachid,'topicCategory':catlist})
    tmpdf = pd.DataFrame(tmplist)  
    tmpsingledf = allresults[['_id','topicCategory']].loc[allresults['_id'].isin(singles['_id'].tolist())]
    idlist = tmpsingledf['_id'].tolist()
    catlist = tmpsingledf['topicCategory'].tolist()
    cattycat = [[x] for x in catlist]
    list_of_tuples = list(zip(idlist,cattycat))
    singledf = pd.DataFrame(list_of_tuples, columns = ['_id', 'topicCategory']) 
    cleanresults = pd.concat((tmpdf,singledf),ignore_index=True)
    return(cleanresults)

In [8]:
training_dict = generate_training_dict()
trainingdf = transform_training_dict(training_dict)
cleanresults = clean_results(trainingdf)
cleanresults.to_json(os.path.join(RESULTSPATH,'topicCats.json'),orient='records')

## Manual inspection
This section contains script that was run for the purposes of manually inspecting the data in order to determine suitable default cut off points and threshholds.

### How to determine if a keyword is suitable for inclusion on a topicCategory-specific list
1. Verify that a more generic version of the word is not already available: ie- if the keyword is 'manic depression', but the term 'depression' is already on the list, it should not be necessary to add 'manic depression'
2. If a more generic version of the word is not already available, search the term on outbreak.info/resources to see if appropriate results pop up.
3. If criteria 2 is met, perform a litcovid search for the term and scan the titles and abstracts. The top 10 most relevant results should be within topic, and 4/5 of the least relevant results should also be within topic.
4. Also check that 12/20 of the most recent results are also within topic. This is to help account for the creation of off-topic new terms/phrases which may enclose the keyword, or changes in connotations based on trends.
5. If criteria 1-4 are met, add the term to the list


#### Main script for manual inspection

In [ ]:
genkw = get_generic_terms()
cat_dict = load_category_ids()
for everytopic in cat_dict.keys():
    kwfreq = get_kw_freq(everytopic)
    kwfreq = negate_genkw(kwfreq,genkw)
    export_most_freqkw(kwfreq,everytopic,toppercent=1)

In [ ]:
#### Expand on keywordlist pulling the keywords for each id, generate frequency list
#### Verify that most common conceptually related keywords are in the list

## fetch the keywords for every pmid in the environment list
environmentdf = batch_fetch_kw(cat_dict['Environment'])

## unstack the dataframe and lower case the keywords for consistency
envkwdf = environmentdf.set_index('_id').keywords.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index()
envkwdf.rename(columns={0:'keywords'},inplace=True)
envkwdf['keywords']=envkwdf['keywords'].astype(str).str.lower()

## Determine keyword frequency
kwfreq = envkwdf.groupby('keywords').size().reset_index(name='count')
kwfreq.sort_values('count',ascending=False,inplace=True)
print(len(kwfreq))

#### Inspect number of words within various frequency categories
## Identify length of keywords short enough for review
min20 = kwfreq.loc[kwfreq['count']>20]
min10 = kwfreq.loc[kwfreq['count']>10]
min5 = kwfreq.loc[kwfreq['count']>5]
min3 = kwfreq.loc[kwfreq['count']>3]
min2 = kwfreq.loc[kwfreq['count']>2]
print(len(min20),len(min10),len(min5),len(min3),len(min2))

#### export results and inspect for potential keywords that are specific enough to this topicCategory
## Do not use words that may be frequently used outside of this topicCategory
## Eg- Behavior: 'stress' is too generic, may refer to oxidative stress
## Eg- Environment: 'climate' is too generic, may refer to current trends or the work environment
## In this case, we're going with words that were mentioned by at least 3 pmids (ie- ~top 5% most frequent words)
min3.to_csv(os.path.join(RESULTSPATH,'wordfrequencies/environment_min3.txt'),sep='\t',header=True)

In [ ]:
## fetch the keywords for every pmid in the environment list
behavedf = batch_fetch_kw(cat_dict['Behavioral Research'])

## unstack the dataframe and lower case the keywords for consistency
bhkwdf = behavedf.set_index('_id').keywords.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index()
bhkwdf.rename(columns={0:'keywords'},inplace=True)
bhkwdf['keywords']=bhkwdf['keywords'].astype(str).str.lower()

## Determine keyword frequency
kwfreq = bhkwdf.groupby('keywords').size().reset_index(name='count')
kwfreq.sort_values('count',ascending=False,inplace=True)

print(len(kwfreq))

#### Inspect number of words within various frequency categories
## Identify length of keywords short enough for review
min20 = kwfreq.loc[kwfreq['count']>20]
min10 = kwfreq.loc[kwfreq['count']>10]
min7 = kwfreq.loc[kwfreq['count']>7]
min5 = kwfreq.loc[kwfreq['count']>5]
min3 = kwfreq.loc[kwfreq['count']>3]
min2 = kwfreq.loc[kwfreq['count']>2]
print(len(min20),len(min10),len(min7),len(min5),len(min3),len(min2))

#### export results and inspect for potential keywords that are specific enough to this topicCategory
## Do not use words that may be frequently used outside of this topicCategory
## Eg- Behavior: 'stress' is too generic, may refer to oxidative stress
## Eg- Environment: 'climate' is too generic, may refer to current trends or the work environment
## In this case, we're going with words that were mentioned by at least 7 pmids (ie- ~top 5% most frequent words)
min7.to_csv(os.path.join(RESULTSPATH,'wordfrequencies/behavior_min7.txt'),sep='\t',header=True)